In [1]:
import pandas as pd
import numpy as np
import time
import datetime
import pickle
from multiprocessing import Pool
from tqdm import tqdm
tqdm.monitor_interval = 0
#data_for_test = pd.read_csv('../ForecastDataforTesting_20171124.csv')

def find_the_path(param):
    id, weather_map, day, start_x, start_y, end_x, end_y, x_min, x_max, y_min, y_max = param
    # 超过15坠毁

    def evaluate_weather(x, y, point):
        weather = weather_map[day][str(point // 30 + 3)][x][y]
        return weather

    def calculate_value(p_after, point):
        return p_after*(2*point-1440)

    wind_block = 0.05
    stop = False

    # 绘制结束可选状态
    # 最小完成时间
    t_least = 2 * (abs(start_x - end_x) + abs(start_y - end_y)) // 60
    # 倒推最小路径
    # 获取可用路径
    t_available = []
    for t in range(3, 21):
        if t - 3 >= t_least and weather_map[day][str(t)][end_x][end_y] > wind_block:
            t_available.append(t)
    # 遍历关键节点路径
    if len(t_available) == 0:
        print('no end points available!')
        return 0
    elif len(t_available) == 1:
        pass
    else:
        temp = []
        for i in range(0, len(t_available) - 1):
            if t_available[i] - t_available[i + 1] == -1:
                continue
            else:
                temp.append(t_available[i])
        temp.append(t_available[-1])
        t_available = temp

    routes = []
    print(t_available)
    available_start = []
    for t in t_available:
        print(t)
        # 设置存储变量初始值
        stop = False
        t_start = (t-2)*30-1
        legal_state = {}
        value_max = {}
        legal_state[str(t_start)] = set()
        legal_state[str(t_start)].add((end_x, end_y))
        # p_after contains this point prob
        value_max[(end_x, end_y, t_start)] = {'max_value':calculate_value(evaluate_weather(end_x,end_y,t_start),t_start),
                                              'p_after':evaluate_weather(end_x,end_y,t_start),'end_point':t_start, 'routine':[(end_x,end_y,t_start)]}
        # 设置迭代程序
        for point in tqdm(range(0, t_start)[::-1]):
            #renew_state(legal_state[str(point + 1)], point)
            last_state = legal_state[str(point+1)]
            legal_state[str(point)] = set()
            for item in last_state:
                # 节约计算时间
                if abs(item[0] - start_x) + abs(item[1] - start_y) > point:  # 还有改进空间
                    continue
                new_state = [(item[0], item[1]), (item[0] + 1, item[1]), (item[0] - 1, item[1]), (item[0], item[1] + 1),
                             (item[0], item[1] - 1)]
                for next_state in new_state:
                    # 是否在指定区域内
                    if x_min <= next_state[0] <= x_max and y_min <= next_state[1] <= y_max:
                        # 之前是否保存过
                        if next_state not in legal_state[str(point)]:
                            # 天气是否合格
                            if weather_map[day][str(point // 30 + 3)][next_state[0]][next_state[1]] > wind_block:
                                legal_state[str(point)].add(next_state)
                                # 判断某个时刻的legal_state是否为空
            if len(legal_state[str(point)]) == 0:
                print(id, day, t, 'legal state is null')
                break
            for item in legal_state[str(point)]:
                if item[0] == end_x and item[1] == end_y:
                    wea_end = evaluate_weather(end_x,end_y,point)
                    value_max[(end_x, end_y, point)] = {'max_value':calculate_value(wea_end,point),'p_after': wea_end , 'end_point': point,'routine':[(end_x,end_y,point)]}
                else:
                    temp = []
                    old_state = [(item[0], item[1]), (item[0] + 1, item[1]), (item[0] - 1, item[1]),
                                 (item[0], item[1] + 1),
                                 (item[0], item[1] - 1)]
                    for o_s in old_state:
                        if o_s in last_state:
                            temp.append((value_max[(o_s[0], o_s[1], point + 1)]['max_value'],(o_s[0], o_s[1], point + 1)))

                    temp.sort(key=lambda x: x[0])

                    # 同一时间段同一个点
                    if item[0]==temp[0][1][0] and item[1]==temp[0][1][1] and point%30!=29:
                        current_node = [(item[0], item[1], point)]
                        current_node.extend(value_max[temp[0][1]]['routine'])
                        value_max[(item[0], item[1], point)] = {'max_value':temp[0][0],'p_after':value_max[temp[0][1]]['p_after'],
                                                                'end_point':value_max[temp[0][1]]['end_point'],'routine':current_node}
                    else:
                        #p = evaluate_weather(temp[0][1][0], temp[0][1][1], point)
                        p = evaluate_weather(item[0], item[1], point)
                        p_after = p*value_max[temp[0][1]]['p_after']
                        # min(1440 +(2*n-1440)*Fj*pj)
                        new_end_point = value_max[temp[0][1]]['end_point']
                        max_value = calculate_value(p_after, new_end_point)
                        current_node = [(item[0], item[1], point)]
                        current_node.extend(value_max[temp[0][1]]['routine'])
                        value_max[(item[0], item[1], point)] = {'max_value':max_value,'p_after':p_after,'end_point':new_end_point,'routine':current_node}
                    # release memory
            if point<t_start-4:
                del legal_state[str(point + 3)]
                for item in legal_state[str(point+2)]:
                    if item[0]==start_x and item[1]==start_y:
                        continue
                    else:
                        del value_max[(item[0],item[1],point+2)]

                # return 0
        # if stop == True:
        #     continue

        # 寻找最优路径
        for k in range(0, t_start):
            try:
                available_start.append((k,value_max[(start_x,start_y,k)]['max_value'],value_max[(start_x,start_y,k)]['routine']))
            except:
                continue

    available_start.sort(key=lambda x:x[1])
    if len(available_start)==0:
        print(id,int(day)+5,'failed')
        return 0
    elif len(available_start)>60:
        print(id, int(day) + 5, 'success')
        return(id, int(day) + 5, available_start[0:60])
    else:
        print(id, int(day) + 5, 'success')
        return (id, int(day) + 5,available_start)

In [2]:
citys = pd.read_csv('CityData.csv')
#file_weather = open('weather_online_output_1227_h_p.pkl', 'rb')
file_weather = open('layer_1_output_prob_final.pkl', 'rb')
#file_weather = open('weather_guanyang.pkl', 'rb')
weather_map = pickle.load(file_weather)
# parameters for the solutions
result = []
params = []
for day in range(6, 11):
    start_x = citys.loc[0, 'xid'] - 1
    start_y = citys.loc[0, 'yid'] - 1
    end_points = np.array(citys.loc[1::, :])
    x_max = 548 - 1
    x_min = 1 - 1
    y_max = 421 - 1
    y_min = 1 - 1
    # max_hour = 8
    # t_max = int(max_hour * 60 / 2)  # （18h）*（60min/h）/（2min/格）

    for dest in end_points:
        id = dest[0]
        end_x = dest[1]-1
        end_y = dest[2]-1
        params.append((id, weather_map, str(day-5), start_x, start_y, end_x, end_y, x_min, x_max, y_min, y_max))
print(len(params))
pool = Pool(8)
#print(params)
result = pool.map(find_the_path, params)
pool.close()  # 关闭进程池，不再接受新的进程
pool.join()  # 主进程阻塞等待子进程的退出


50
[20]


  0%|          | 0/539 [00:00<?, ?it/s]

20
[20]


  0%|          | 0/539 [00:00<?, ?it/s]

20


...

6 10 success


100%|██████████| 539/539 [25:23<00:00,  2.83s/it]


10 10 success


100%|██████████| 539/539 [23:27<00:00,  2.61s/it]


4 10 success


In [3]:
for item in result:
    try:
        n = len(item[2])
        available_value = [(item[2][i][1],item[2][i][0]) for i in range(n)]
    
        print(item[1],item[0])
        for jj in available_value:
            print(jj)
    except:
        print('something wrong',item)

6 1
(-600.380859375, 30)
(-600.380859375, 31)
(-600.380859375, 32)
(-600.380859375, 33)
(-600.380859375, 34)
(-600.380859375, 35)
(-600.380859375, 36)
(-600.380859375, 37)
(-600.380859375, 38)
(-600.380859375, 39)
(-600.380859375, 40)
(-600.380859375, 41)
(-599.58984375, 42)
(-598.798828125, 43)
(-598.4033203125, 44)
(-597.216796875, 45)
(-596.0302734375, 46)
(-594.580078125, 0)
(-594.4482421875, 47)
(-592.8662109375, 48)
(-590.4931640625, 49)
(-589.306640625, 50)
(-588.515625, 1)
(-588.515625, 51)
(-588.1201171875, 52)
(-587.724609375, 53)
(-587.546875, 2)
(-587.546875, 3)
(-587.3291015625, 54)
(-586.93359375, 55)
(-586.5380859375, 56)
(-586.142578125, 57)
(-585.7470703125, 58)
(-585.125, 4)
(-581.0009765625, 29)
(-573.984375, 5)
(-572.6953125, 59)
(-571.5087890625, 28)
(-565.75, 6)
(-562.8076171875, 27)
(-560.90625, 7)
(-558.484375, 8)
(-556.4794921875, 10)
(-556.4794921875, 11)
(-556.4794921875, 12)
(-556.4794921875, 13)
(-556.4794921875, 14)
(-556.4794921875, 15)
(-556.4794921875, 

In [36]:
for item in result:
    if item[0]==8 and item[1]==6:
        for i in item[2]:
            print(i[2][0],len(i[2]))
            

(141, 327, 30) 420
(141, 327, 30) 420
(141, 327, 31) 419
(141, 327, 31) 419
(141, 327, 32) 418
(141, 327, 32) 418
(141, 327, 33) 417
(141, 327, 33) 417
(141, 327, 34) 416
(141, 327, 35) 415
(141, 327, 34) 416
(141, 327, 35) 415
(141, 327, 36) 414
(141, 327, 37) 413
(141, 327, 36) 414
(141, 327, 37) 413
(141, 327, 38) 412
(141, 327, 39) 411
(141, 327, 38) 412
(141, 327, 39) 411
(141, 327, 40) 410
(141, 327, 40) 410
(141, 327, 41) 409
(141, 327, 41) 409
(141, 327, 42) 408
(141, 327, 42) 408
(141, 327, 43) 407
(141, 327, 44) 406
(141, 327, 43) 407
(141, 327, 44) 406
(141, 327, 45) 405
(141, 327, 45) 405
(141, 327, 46) 404
(141, 327, 46) 404
(141, 327, 47) 403
(141, 327, 47) 403
(141, 327, 48) 402
(141, 327, 48) 402
(141, 327, 49) 401
(141, 327, 49) 401
(141, 327, 50) 400
(141, 327, 50) 400
(141, 327, 51) 399
(141, 327, 51) 399
(141, 327, 52) 398
(141, 327, 52) 398
(141, 327, 53) 397
(141, 327, 54) 396
(141, 327, 53) 397
(141, 327, 54) 396
(141, 327, 55) 395
(141, 327, 55) 395
(141, 327, 5

TypeError: 'int' object is not subscriptable

In [30]:
# 观察法求最优解
# 采用策略
# waiting:[(day,city_id,route_count,start_point)]
# (6,8),(9,3)
waiting = [(6,2,0,30),(6,9,5,35),(6,3,10,40),(6,1,15,45),(6,8,40,50),(6,5,25,55),(6,6,31,60),
           (6,7,36,65),(6,10,40,70),(6,4,45,75),
          (7,7,0,0),(7,8,5,5),(7,6,10,10),(7,5,11,30),(7,3,15,15),(7,2,5,35),(7,1,10,40),
           (7,9,30,45),(7,10,1,119),(7,4,24,25),
           (8,4,0,0),(8,3,5,5),(8,1,10,10),(8,2,59,89),(8,5,54,84),(8,6,15,15),(8,7,49,79),(8,8,20,20),
           (8,9,25,25),(8,10,30,30),
           (9,10,0,0),(9,9,5,5),(9,8,10,10),(9,7,29,29),(9,6,42,42),(9,5,37,37),(9,4,59,59),
           (9,3,45,15),(9,2,20,20),
           (10,4,0,0),(10,6,5,5),(10,7,10,10),(10,8,15,15),(10,3,20,20),(10,2,25,25),(10,1,30,30),
           (10,5,35,35),(10,9,40,40),(10,10,45,45)
           
          ]


submit_queue = []
for item in result:
    if item!=0:
        for (day,city_id,route_count,start_point) in waiting:
            if item[1]==day and item[0]==city_id:
                submit_queue.append((city_id,day,item[2][route_count][2]))
            else:
                pass
        

In [22]:
waiting = [(6,2,0,30),(6,9,5,35),(6,3,10,40),(6,1,15,45),(6,8,40,50),(6,5,25,55),(6,6,31,60),
           (6,7,36,65),(6,10,40,70),(6,4,45,75),
           (7,7,0,0),(7,8,5,5),(7,6,10,10),(7,5,11,30),(7,3,15,15),(7,2,5,35),(7,1,10,40),
           (7,9,30,45),(7,10,1,119),(7,4,24,25),
           (8,4,0,0),(8,3,5,5),(8,1,10,10),(8,2,59,89),(8,5,54,84),(8,6,15,15),(8,7,49,79),(8,8,20,20),
           (8,9,25,25),(8,10,30,30),
           (9,10,0,0),(9,9,5,5),(9,8,10,10),(9,7,29,29),(9,6,42,42),(9,5,37,37),(9,4,59,59),
           (9,3,45,15),(9,2,20,20),
           (10,4,0,0),(10,6,5,5),(10,7,10,10),(10,8,15,15),(10,3,20,20),(10,2,25,25),(10,1,30,30),
           (10,5,35,35),(10,9,40,40),(10,10,45,45)
           
          ]

In [23]:
result[0]

(1,
 6,
 [(30,
   -600.380859375,
   [(141, 327, 30),
    (141, 327, 31),
    (141, 327, 32),
    (141, 327, 33),
    (141, 327, 34),
    (141, 327, 35),
    (141, 327, 36),
    (141, 327, 37),
    (141, 327, 38),
    (141, 327, 39),
    (141, 327, 40),
    (141, 327, 41),
    (142, 327, 42),
    (142, 326, 43),
    (142, 325, 44),
    (142, 324, 45),
    (142, 323, 46),
    (142, 322, 47),
    (142, 321, 48),
    (142, 320, 49),
    (142, 319, 50),
    (142, 318, 51),
    (142, 317, 52),
    (142, 316, 53),
    (142, 315, 54),
    (142, 314, 55),
    (142, 313, 56),
    (142, 312, 57),
    (142, 311, 58),
    (142, 310, 59),
    (142, 309, 60),
    (142, 308, 61),
    (142, 307, 62),
    (142, 306, 63),
    (142, 305, 64),
    (142, 304, 65),
    (142, 303, 66),
    (142, 302, 67),
    (142, 301, 68),
    (142, 300, 69),
    (143, 300, 70),
    (144, 300, 71),
    (144, 299, 72),
    (144, 299, 73),
    (144, 299, 74),
    (144, 299, 75),
    (144, 299, 76),
    (144, 299, 77),
    (1

In [29]:
# check program
for i in range(6,11):
    i_day = []
    for item in waiting:
        if item[0]==i:
            i_day.append(item[3])
    print(i_day)
    print(solution_constraint(i_day))

[30, 35, 40, 45, 50, 55, 60, 65, 70, 75]
1
[0, 5, 10, 30, 15, 35, 40, 45, 119, 25]
1
[0, 5, 10, 89, 84, 15, 79, 20, 25, 30]
1
[0, 5, 10, 29, 42, 37, 59, 15, 20]
1
[0, 5, 10, 15, 20, 25, 30, 35, 40, 45]
1


In [27]:
# check program
for item in result:
    if item!=0:
        for jtem in waiting:
            if item[0] ==jtem[1] and item[1]==jtem[0]:
                assert item[2][jtem[2]][0]==jtem[3]
           

In [ ]:
# check program
for item in result:
    if item!=0:
        for jtem in waiting:
            if item[0] ==jtem[1] and item[1]==jtem[0]:
                assert item[2][jtem[2]][0]==jtem[3]
           

In [4]:
# 蒙特卡洛求最优解
def solution_constraint(s):
    s.sort()
    sturn = np.diff(s)
    if len(sturn[sturn<5])==0:
        return 1
    else:
        return 0
    
    
value_turn = {}
for item in result:
    if item!=0:
        n = len(item[2])
        available_value = [(item[2][i][1],item[2][i][0]) for i in range(n)]
        try:
            value_turn[str(item[1])].append((item[0],available_value))
        except:
            value_turn[str(item[1])] = [(item[0],available_value)]


In [17]:
import pickle
a ={}
a['test'] = result
pickle.dump(a,open('submit_file_final.pkl','wb'))


In [5]:
# Montecaro Methood
import random
import time
best_solution = {}
for (key,values) in value_turn.items():
    print(key)
    iter_num = 0
    id_total = len(values)
    current_loss = 0
    n = 1000000
    t1 = time.time()
    while iter_num<n:
        iter_num += 1
        waiting_solution = np.zeros((id_total,4))
        i = 0
        for item in values:
            #upper_limit = np.random.choice([30,len(item[1])-1], p=[0.8, 0.2])
            count_by_order = random.randint(0,len(item[1])-1)
            (loss,start_point) = item[1][count_by_order]
            waiting_solution[i,:] = [item[0],loss,start_point,count_by_order]
            i += 1
        if solution_constraint(waiting_solution[:,2]):
            loss = np.sum(waiting_solution[:,1])
            #print(loss,current_loss)
            if loss<current_loss:
                current_loss = loss
                best_solution[key] = waiting_solution
                print(iter_num,' current loss %f'%(current_loss))
            else:
                continue
        else:
            continue
    t2 = time.time()
    print(t2-t1)
    
        

6
933  current loss -6278.997803
1954  current loss -6344.641602
17735  current loss -6375.451416
40736  current loss -6383.685303
58989  current loss -6407.434326
155659  current loss -6417.049561
315803  current loss -6447.485107
443461  current loss -6453.636475
36.07171320915222
10
36276  current loss -7584.644043
177734  current loss -8064.415039
183523  current loss -8424.516602
36.43015384674072
7
102  current loss -6100.657227
1107  current loss -6153.981934
1441  current loss -6494.860840
1891  current loss -6749.623535
147115  current loss -6808.794434
35.60881018638611
9
7404  current loss -6502.700684
16602  current loss -6503.880371
18296  current loss -6600.817383
22605  current loss -6713.880371
151554  current loss -6714.884766
292014  current loss -6724.581543
332552  current loss -6741.825195
33.74972653388977
8
399  current loss -7470.768555
2805  current loss -7486.576172
3093  current loss -7509.064453
5336  current loss -7516.697266
7367  current loss -7558.894043

In [6]:
submit_queue = []
waiting = []
for (a,b) in best_solution.items():
    for i in range(len(b)):
        waiting.append((int(a),int(b[i,0]),int(b[i,3])))
print(waiting)
for item in result:
    if item!=0:
        for (day,city_id,route_count) in waiting:
            if item[1]==day and item[0]==city_id:
                submit_queue.append((city_id,day,item[2][route_count][2]))
            else:
                pass

[(6, 1, 45), (6, 2, 5), (6, 3, 11), (6, 4, 46), (6, 5, 31), (6, 6, 41), (6, 7, 55), (6, 8, 42), (6, 9, 0), (6, 10, 35), (10, 1, 26), (10, 2, 42), (10, 3, 8), (10, 4, 1), (10, 5, 36), (10, 6, 59), (10, 7, 47), (10, 8, 21), (10, 9, 16), (10, 10, 31), (7, 1, 27), (7, 2, 7), (7, 3, 13), (7, 4, 8), (7, 5, 0), (7, 6, 49), (7, 7, 8), (7, 8, 20), (7, 9, 4), (7, 10, 3), (8, 1, 10), (8, 2, 54), (8, 3, 0), (8, 4, 28), (8, 5, 41), (8, 6, 35), (8, 7, 49), (8, 8, 15), (8, 9, 5), (8, 10, 40), (9, 2, 8), (9, 3, 40), (9, 4, 53), (9, 5, 33), (9, 6, 59), (9, 7, 40), (9, 8, 18), (9, 9, 24), (9, 10, 0)]


In [11]:
submit_queue

[(1,
  6,
  [(141, 327, 0),
   (141, 326, 1),
   (141, 325, 2),
   (141, 324, 3),
   (140, 324, 4),
   (139, 324, 5),
   (139, 323, 6),
   (138, 323, 7),
   (138, 322, 8),
   (137, 322, 9),
   (136, 322, 10),
   (135, 322, 11),
   (134, 322, 12),
   (133, 322, 13),
   (132, 322, 14),
   (131, 322, 15),
   (131, 321, 16),
   (131, 320, 17),
   (131, 319, 18),
   (131, 318, 19),
   (131, 317, 20),
   (131, 316, 21),
   (130, 316, 22),
   (129, 316, 23),
   (128, 316, 24),
   (127, 316, 25),
   (126, 316, 26),
   (125, 316, 27),
   (125, 315, 28),
   (124, 315, 29),
   (123, 315, 30),
   (122, 315, 31),
   (121, 315, 32),
   (120, 315, 33),
   (119, 315, 34),
   (118, 315, 35),
   (117, 315, 36),
   (116, 315, 37),
   (115, 315, 38),
   (114, 315, 39),
   (113, 315, 40),
   (112, 315, 41),
   (111, 315, 42),
   (110, 315, 43),
   (109, 315, 44),
   (108, 315, 45),
   (107, 315, 46),
   (106, 315, 47),
   (105, 315, 48),
   (104, 315, 49),
   (103, 315, 50),
   (102, 315, 51),
   (101, 315

In [31]:
#routes = find_the_path()
#result.append([id, day, routes])
print(len(result))
# 制作提交文件格式
submit = []
base_time = "2017-12-10 03:00:00"
base_time = datetime.datetime.strptime(base_time, "%Y-%m-%d %H:%M:%S")
#item (id,day,routine)
for item in submit_queue:
    if item != 0:
        time = base_time
        for point in item[2]:
            time_point = base_time+datetime.timedelta(minutes=2)*point[2]

            time_format = time_point.strftime('%H:%M')
            submit.append([item[0], item[1], time_format, point[0]+1, point[1]+1])
file_data = pd.DataFrame(submit)
print(file_data)
file_data.to_csv('submit_file_final_0209_2.csv', index=False, header=False)

50
        0   1      2    3    4
0       1   6  04:30  142  328
1       1   6  04:32  142  327
2       1   6  04:34  142  326
3       1   6  04:36  142  325
4       1   6  04:38  142  324
5       1   6  04:40  142  323
6       1   6  04:42  142  322
7       1   6  04:44  142  321
8       1   6  04:46  142  320
9       1   6  04:48  142  319
10      1   6  04:50  142  318
11      1   6  04:52  142  317
12      1   6  04:54  142  316
13      1   6  04:56  142  315
14      1   6  04:58  142  314
15      1   6  05:00  142  313
16      1   6  05:02  142  312
17      1   6  05:04  142  311
18      1   6  05:06  142  310
19      1   6  05:08  143  310
20      1   6  05:10  143  309
21      1   6  05:12  143  308
22      1   6  05:14  143  307
23      1   6  05:16  143  306
24      1   6  05:18  143  305
25      1   6  05:20  143  304
26      1   6  05:22  143  303
27      1   6  05:24  143  302
28      1   6  05:26  143  301
29      1   6  05:28  144  301
...    ..  ..    ...  ...  ...
11570